<h1>Getting Started with PySC2</h1>
<p>For additional information, please refer to the official GitHub repository linked <a href="https://github.com/deepmind/pysc2">here</a>.</p>
<p>To get started with PySC2, first verify that you have the proper Python packages installed as well as the mini game maps linked in the PySC2 repository!</p>
<p>Please run the following cell to make sure that your computer has all of the proper software setup.</p>
<p>If you have everything installed properly, the running the following cell should open an instance of StarCraft with a randomly acting agent playing on the MoveToBeacon map:</p>
<div style="background-color:#300a24"><b><p style="color:white">python3 -m pysc2.bin.agent --map MoveToBeacon --agent pysc2.agents.random_agent.RandomAgent</p></b></div>

<p>Assuming all of that worked, we will continue by setting up some of the basic configuration for our StarCraft AI. In order to make our AI, we will create a Python class to represent our StarCraft agent. We will make our class inherit from the base agent which PySC2 provides.</p>

<p>Some things worth noting are that we define a list of default actions for our agent to use. This will be used to restrict which actions our agent is allowed to perform. To view the list of all valid actions our agent can perform, try entering the following command in a terminal:</p>
<div style="background-color:#300a24"><b><p style="color:white">python3 -m pysc2.bin.valid_actions --hide_specific</p></b></div>


<p>Running this command produces many lines of output giving you the numerical id's of various StarCraft actions our agent can perform. A smaller subset of actions has been selected with comments with their name so that our agent is not slowed down with too many actions to learn.<p>

In [1]:
from pysc2.agents.base_agent import BaseAgent
from pysc2.lib import actions

default_actions = [
   0, #no_op                                              ()
   1, #move_camera                                        (1/minimap [64, 64])
   2, #select_point                                       (6/select_point_act [4]; 0/screen [84, 84])
   3, #select_rect                                        (7/select_add [2]; 0/screen [84, 84]; 2/screen2 [84, 84])
   4, #select_control_group                               (4/control_group_act [5]; 5/control_group_id [10])
   5, #select_unit                                        (8/select_unit_act [4]; 9/select_unit_id [500])
   6, #select_idle_worker                                 (10/select_worker [4])
   7, #select_army                                        (7/select_add [2])
   8, #select_warp_gates                                  (7/select_add [2])
   9, #select_larva                                       ()
  10, #unload                                             (12/unload_id [500])
  11, #build_queue                                        (11/build_queue_id [10])
  12, #Attack_screen                                      (3/queued [2]; 0/screen [84, 84])
  13 #Attack_minimap                                     (3/queued [2]; 1/minimap [64, 64])
]

#This represents the base interface for how our agent will work
#We separate this from the StarCraft II agent class so we can focus on the underlying RL
#implementation later...
class Brain:
    def __init__(self, race="T", actions = default_actions):
        self.race = race
        self.actions = actions
    
    #By default, our brain will just do nothing.
    #We will change this later...
    def step(self, obs):
        return 0, []


#This represents the actual agent which will play StarCraft II
class MyAgent(BaseAgent):
    def __init__(self, brain = Brain()):
        super().__init__() #call parent constructor
        assert isinstance(brain, Brain)
        self.brain = brain
        
    def step(self, obs): #This function is called once per frame to give the AI observation data and return its action
        super().step(obs) #call parent base method
        action, params = self.brain.step(obs)
        return actions.FunctionCall(action, params)
        
agent = MyAgent()

<p>From here, we can test our our agent by calling the following cell. The first line exports our notbook code as a Python file. The second line actually runs our agent.</p>

<div style="background-color:#300a24"><b><p style="color:white">
jupyter nbconvert --to script PySC2_Basics<br>python3 -m pysc2.bin.agent --map MoveToBeacon --agent PySC2_Basics.MyAgent</p></b></div>

<p>To scale up our training performance, we will be using the Synchronous Actor Advantage Critic (A2C) reinforcement learning algorithm, which allows us to train our agent multiple times in parallel. Starter code is provided <a href="https://github.com/MG2033/A2C">here</a>. First, we will need to import some modules.<p>

In [2]:
import tensorflow as tf
import numpy as np

In [6]:
default_actor_optimizer = tf.train.RMSPropOptimizer(0.0001, name='RMSPropA')
default_critic_optimizer = tf.train.RMSPropOptimizer(0.001, name='RMSPropC')
UNIT_ELEMENTS = 7
ENTROPY_BETA = 0.01
default_nonspatial_spec = 10

class StateNet:
    def __init__(self, scope, nonspatial_actions,
                 resolution=84, channels=20, max_multi_select=10,
                 max_cargo=10, max_build_queue=10, l2_scale=0.01, hidden_size=256, lstm_size=256):
        with tf.variable_scope(scope):
            self.structured_observation = tf.placeholder(tf.float32, [None, 11], 'StructuredObservation')
            self.single_select = tf.placeholder(tf.float32, [None, 1, UNIT_ELEMENTS], 'SingleSelect')
            self.cargo = tf.placeholder(tf.float32, [None,  max_cargo, UNIT_ELEMENTS], 'Cargo')
            self.multi_select = tf.placeholder(tf.float32, [None, max_multi_select, UNIT_ELEMENTS], 'Multiselect')
            self.build_queue = tf.placeholder(tf.float32, [None,  max_build_queue, UNIT_ELEMENTS], 'BuildQueue')
            self.units = tf.concat([self.single_select,
                                    self.multi_select,
                                    self.cargo,
                                    self.build_queue], axis=1)
            print('Units shape:',self.units.shape)
            self.control_groups = tf.placeholder(tf.float32, [None, 10, 2], 'ControlGroups')
            self.available_actions = tf.placeholder(tf.float32, [None, nonspatial_actions], 'AvailableActions')
            self.used_actions = tf.placeholder(tf.float32, [None, nonspatial_actions], 'UsedActions')
            self.actions = tf.concat([self.available_actions,
                                      self.used_actions], axis=1)
            print('Actions shape:', self.actions.shape)
            self.nonspatial_features = tf.concat([
                    self.structured_observation,
                    tf.reshape(self.units, [-1, UNIT_ELEMENTS * (1+max_multi_select+max_build_queue+max_cargo)]),
                    tf.reshape(self.control_groups, [-1, 10*2]),
                    tf.reshape(self.actions, [-1, 2 * nonspatial_actions])
                ], axis=1)
            
            print('Nonspatial features shape:',self.nonspatial_features.shape)
            
            self.spatial_features = tf.placeholder(tf.float32, [None, resolution, resolution, channels], 'SpatialFeatures')
            self.conv1 = tf.layers.conv2d(inputs=self.spatial_features, filters=32,
                                          kernel_size=[5, 5],
                                          kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_scale),
                                          activation=tf.nn.relu, name='Convolutional1')
            print('Convolutional1 Shape:',self.conv1.shape)
            self.max_pool1 = tf.layers.max_pooling2d(inputs=self.conv1, pool_size=[2, 2], strides=2, name='Pool1')
            print('Pool Shape1:',self.max_pool1.shape)
            self.conv2 = tf.layers.conv2d(inputs=self.max_pool1, filters=64,
                                          kernel_size=[5, 5],
                                          kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_scale),
                                          activation=tf.nn.relu, name='Convolutional2')
            print('Convolutional2 Shape:',self.conv2.shape)
            self.max_pool2 = tf.layers.max_pooling2d(inputs=self.conv2, pool_size=[2, 2], strides=2, name='Pool2')
            print('Pool Shape2:',self.max_pool2.shape)
            self.max_pool2_flat = tf.reshape(self.max_pool2, [-1, 18 * 18 * 64])
            print('Pool flattened shape:', self.max_pool2_flat.shape)
            self.state_flattened = tf.concat([self.max_pool2_flat, self.nonspatial_features], 1, name='StateFlattened')
            print('State flat shape:', self.state_flattened.shape)
            self.hidden_1 = tf.layers.dense(self.state_flattened, hidden_size, tf.nn.relu, name='Hidden1')

tf.reset_default_graph()
test_state = StateNet('global', default_nonspatial_spec)



Units shape: (?, 31, 7)
Actions shape: (?, 20)
Nonspatial features shape: (?, 268)
Convolutional1 Shape: (?, 80, 80, 32)
Pool Shape1: (?, 40, 40, 32)
Convolutional2 Shape: (?, 36, 36, 64)
Pool Shape2: (?, 18, 18, 64)
Pool flattened shape: (?, 20736)
State flat shape: (?, 21004)


In [ ]:
class ACNet:
    def __init__(self, scope, state_spec = default_state_spec,
                 nonspatial_spec = default_nonspatial_spec,
                 spatial_spec = default_spatial_spec,
                 global_ACNet=None,
                 actor_optimizer=default_actor_optimizer,
                 critic_optimizer=default_critic_optimizer):
        with tf.variable_scope(scope):
            self.state = tf.placeholder(tf.float32, [None]+[i for i in state_spec], 'State')    
            self.build_actor_critic(scope, nonspatial_spec, resolution)
            if global_ACNet is not None:# We have a global net, making a worker net
                self.nonspatial_action_history = tf.placeholder(tf.float32, [None, nonspatial_spec], 'Action')        # action
                self.v_target = tf.placeholder(tf.float32, [None, 1], 'Vtarget') # v_target value
                td = tf.subtract(self.v_target, self.v, name='TD_error')
                with tf.name_scope('critic_loss'):
                    self.critic_loss = tf.reduce_mean(tf.square(td))

                normal_dist = {}
                log_prob = {}
                self.exp_v = {}
                self.action_loss = 0
                for dim in self.mu:
                    with tf.name_scope('wrap_a_out'):
                        self.mu[dim], self.sigma[dim] = self.mu[dim] * spatial_spec[1], self.sigma[dim] + 1e-4
                    normal_dist[dim] = tf.contrib.distributions.Normal(self.mu[dim], self.sigma[dim])
    
                    with tf.name_scope('a_loss'):
                        log_prob[dim] = normal_dist[dim].log_prob(self.nonspatial_action_history)
                        exp_v = log_prob[dim] * td
                        entropy = normal_dist[dim].entropy()  # encourage exploration
                        self.exp_v = ENTROPY_BETA * entropy + exp_v
                        self.action_loss -= self.exp_v[dim]
    
    def build_actor_critic(self, scope, nonspatial_spec, spatial_spec, actor_units=200, critic_units=100):
        w_init = tf.random_normal_initializer(0., .1)
        with tf.variable_scope('actor'):
            actor_layer = tf.layers.dense(self.state, actor_units, tf.nn.relu6, kernel_initializer=w_init, name='actorlayer')
            # estimated action value
            self.mu = {}
            # estimated variance
            self.sigma = {}
            for dim in ['x', 'y', 'nonspatial']:
                with tf.variable_scope(dim):
                    spec = nonspatial_spec if dim == 'nonspatial' else spatial_spec
                    self.mu[dim] = tf.layers.dense(actor_layer, spec, tf.nn.tanh, kernel_initializer=w_init, name='mu')
                    self.sigma[dim] = tf.layers.dense(actor_layer, spec, tf.nn.softplus, kernel_initializer=w_init, name='sigma')
            
        with tf.variable_scope('critic'):
            critic_layer = tf.layers.dense(self.state, critic_units, tf.nn.relu6, kernel_initializer=w_init, name='criticlayer')
            self.v = tf.layers.dense(critic_layer, 1, kernel_initializer=w_init, name='v')  # estimated value for state
        self.nonspatial_actor_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/actor')
        self.critic_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/critic')
        
        
        
tf.reset_default_graph()
test_global = ACNet('global_ac')
test_worker = ACNet('worker0', global_ACNet=test_global)